In [ ]:
import os
import librosa
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [ ]:
def preprocess_log_mel_spectrogram(file_path, n_mels=128, duration=5.0):
    # Load the audio file
    waveform, sr = librosa.load(file_path, duration=duration)
    if len(waveform) == 0:
            # Skip empty audio files
            return None
    
    hop_length = int(sr * (0.02 - 0.01))
    
    # Extract the Log-Mel-spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(waveform, sr=sr, n_mels=n_mels, hop_length=hop_length)
    log_mel_spectrogram = librosa.amplitude_to_db(mel_spectrogram, ref=np.max)
    
    return log_mel_spectrogram


In [ ]:
wake_word_dir = './wake_word/'
negative_samples_dir = './neg_samples/'


In [ ]:
features = []
labels = []

In [ ]:
%%capture
wake_word_files = os.listdir(wake_word_dir)
for file in wake_word_files:
    file_path = os.path.join(wake_word_dir, file)
    feature = preprocess_log_mel_spectrogram(file_path)
    features.append(feature)
    labels.append(1)  # 1 represents the wake word

In [ ]:
%%capture
negative_samples_files = os.listdir(negative_samples_dir)
for file in negative_samples_files:
    file_path = os.path.join(negative_samples_dir, file)
    feature = preprocess_log_mel_spectrogram(file_path)
    features.append(feature)
    labels.append(0)

In [ ]:
max_shape = max(feature.shape for feature in features)

padded_features = []
for feature in features:
    pad_width = [(0, max_shape[0] - feature.shape[0]), (0, max_shape[1] - feature.shape[1])]
    padded_feature = np.pad(feature, pad_width, mode='constant')
    padded_features.append(padded_feature)

In [ ]:
x = np.array(padded_features)
y = np.array(labels)

In [ ]:
from sklearn.model_selection import train_test_split


#két részre osztom az dathalmaz, az *_trainval-ba mennek a tanító+valikdációs adatok, a *_test-be a tesztadatok
x_trainval, x_test, y_trainval, y_test = train_test_split(x, y, test_size=0.15, random_state=42)
#x_train, x_valid, y_train, y_valid, x_test,y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#ezt követően a tanító + validációs adatokat is két részrte osztom ezzel megkapva a 3 datasetet
x_train, x_valid, y_train, y_valid = train_test_split(x_trainval, y_trainval, test_size=0.1765, random_state=42)


In [ ]:
x_train = np.reshape(x_train, (*x_train.shape, 1))
x_test = np.reshape(x_test, (*x_test.shape, 1))
x_valid = np.reshape(x_valid, (*x_valid.shape, 1))

In [ ]:
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), activation='relu', input_shape=x_train[0].shape))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Flatten())
model1.add(Dense(64, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))

In [ ]:
model1.compile(optimizer='adam',
              #optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('wake_wordv1.h5', monitor='val_loss', save_best_only=True)
es = EarlyStopping(monitor='val_loss', min_delta=0, patience = 5)

callbacks = [checkpoint, es]

In [ ]:
batch_size = 64
hist1 = model1.fit(x_train, y_train,
                 epochs=1000,
                 batch_size=batch_size,
                 validation_data=(x_valid, y_valid),
                 callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt

loss = hist1.history['loss']
val_loss=hist1.history['val_loss']
acc = hist1.history['accuracy']
val_acc = hist1.history['val_accuracy']

epochs = range(len(loss))

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'g', label='Validation loss')
plt.plot(epochs, acc, 'y', label='Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.legend()

plt.show()

In [ ]:
test_loss,test_acc = model1.evaluate(x_test)
print('Training Acc: ',acc[-1])
print('Validation Acc: ', val_acc[-1])
print('Test Loss: ',test_loss)
print('Test Acc: ',test_acc)